[Filecoin has support for JSON-RPC API](https://filecoin-shipyard.github.io/js-lotus-client/intro/json-rpc.html), a standardized way to encode remote procedure calls in JSON. Projects such as Bitcoin and Ethereum have adopted it for use in their APIs.

For example, when you run the Lotus node with lotus daemon, it will listen for connections on TCP port 1234 (by default).

Applications may connect to this port and issue JSON-RPC calls. For example, using CURL, if we want to call the Filecoin.Version method, we might do this:

```bash
curl -X POST \
  -H "Content-Type: application/json" \
  --data '{ 
      "jsonrpc": "2.0", 
      "method": "Filecoin.Version", 
      "params": [], 
      "id": 1 
    }' \
  http://127.0.0.1:1234/rpc/v0 
```

Since Filecoin doesn't have a Python client and tooling around CAR files is still in development, we'll use [the JSON-RPC API to get data from the Filecoin chain](https://lotus.filecoin.io/reference/basics/api-access/).

In [1]:
import requests
import pandas as pd

In [2]:
ENDPOINT_URL = "https://api.node.glif.io"  # Another alternative is "https://lotus.miner.report/mainnet_api/0/node/rpc/v0"

In [3]:
def filecoin_rpc(method: str, params):
    """
    Call a Filecoin JSON-RPC method with the given parameters.
    """
    payload = {
        "jsonrpc": "2.0",
        "method": method,
        "params": params,
        "id": 1,
    }
    response = requests.post(ENDPOINT_URL, json=payload).json()
    if "error" in response:
        raise Exception(response["error"])
    return response["result"]


def get_tipset_by_height(height: int):
    """
    Get a tipset by its height.
    """
    return filecoin_rpc("Filecoin.ChainGetTipSetByHeight", [height, None])

With this we're ready to go. First, let's see the head of the chain:

In [4]:
chain_head = filecoin_rpc("Filecoin.ChainHead", [])
latest_epoch = chain_head["Height"]
print(f"Latest epoch is {latest_epoch}")

Latest epoch is 3010959


In [5]:
pd.json_normalize(chain_head["Blocks"])

,Miner,BeaconEntries,WinPoStProof,Parents,ParentWeight,Height,Timestamp,ForkSignaling,ParentBaseFee,Ticket.VRFProof,ElectionProof.WinCount,ElectionProof.VRFProof,ParentStateRoot./,ParentMessageReceipts./,Messages./,BLSAggregate.Type,BLSAggregate.Data,BlockSig.Type,BlockSig.Data
0,f01372912,"[{'Round': 3106804, 'Data': 'kVnCDgpqregXQEalF...","[{'PoStProof': 3, 'ProofBytes': 'lhXbfO02iyRUB...",[{'/': 'bafy2bzacedmrk3uc22dez5upjlisv4vo4blzm...,71041523330,3010959,1688635170,0,695136275,oL8cu+ZztyIQmJh9vg6oLqmHF5Q+JNopWLci8pE/B5DuPK...,1,hAtTxVDENsSxt7DiFGT+1K1FoWUUyQFIQcBvjzfUfSy5Uy...,bafy2bzaceadxsiykeh2neb2zjxqgaermn435rnmhoe7do...,bafy2bzacebosq7iz7x5zkxshmlfhtivo7lyfjcv4phnfp...,bafy2bzaceadvugsq2dc52nxyvuuhelra4tccsfhdong7s...,2,k0gXt89xmOWL1EP5qeeZVgT79i/0blDpvSb7dS/sO/WV7K...,2,hsMzYbQPJ8TCJSLlydSpoIYXDVIxm+voS7ZxnZf9rP5kuH...
1,f0123261,"[{'Round': 3106804, 'Data': 'kVnCDgpqregXQEalF...","[{'PoStProof': 4, 'ProofBytes': 'mMoJNLRYwUk2t...",[{'/': 'bafy2bzacedmrk3uc22dez5upjlisv4vo4blzm...,71041523330,3010959,1688635170,0,695136275,iDOhFJ5iOlazrLK3Q291zAyl99xNMfElfFoBXNieB/QP2p...,1,oBSdtyuuEiEvdzpzBC8NLO+zRBjY+VisbRUr0NVlRvE4s4...,bafy2bzaceadxsiykeh2neb2zjxqgaermn435rnmhoe7do...,bafy2bzacebosq7iz7x5zkxshmlfhtivo7lyfjcv4phnfp...,bafy2bzaceagc65ow6ywdhyidj2xmmptlom67llbw2pxys...,2,lKcFDsA82rEUXqLXv6KmC/XmASuBHL4/wg/Nuh5xh/roLY...,2,jT4peFYiWWQ1KTWj2vOkKxKGxtvS+1QzHAfEDKu16exwzu...
2,f01771695,"[{'Round': 3106804, 'Data': 'kVnCDgpqregXQEalF...","[{'PoStProof': 3, 'ProofBytes': 'gDHLxZgEhGenk...",[{'/': 'bafy2bzacedmrk3uc22dez5upjlisv4vo4blzm...,71041523330,3010959,1688635170,0,695136275,qri2RfDspLUxb/bWvl97UrN6KlWV7maI8rJAzohncIFqLk...,1,ldE1EQX/APplr1Y4KiBfa4wdPxmqiT3iNsyYi23ryKN32C...,bafy2bzaceadxsiykeh2neb2zjxqgaermn435rnmhoe7do...,bafy2bzacebosq7iz7x5zkxshmlfhtivo7lyfjcv4phnfp...,bafy2bzaced5m4vmmuscypask67r72klxa3caqw2zatbmk...,2,lqOeQulDKKb/7iK2YruDVzrbJfQ1cYNcMaaa/+UMSKGC2R...,2,koxVbQf5vi1HIYiQCoJf27ntXBup1sfuuC6UJ1HjwJTK1s...
3,f01181168,"[{'Round': 3106804, 'Data': 'kVnCDgpqregXQEalF...","[{'PoStProof': 3, 'ProofBytes': 'oeIhKd6dF6I0v...",[{'/': 'bafy2bzacedmrk3uc22dez5upjlisv4vo4blzm...,71041523330,3010959,1688635170,0,695136275,sGocm9/eAQG+6Yn43mfxZ6bML143DCc6sMrog1uu03+9sN...,1,mEaSFWiZFjq2ZZwRTBIlQkW5qDONqQyqIC79kuQ23CHOH7...,bafy2bzaceadxsiykeh2neb2zjxqgaermn435rnmhoe7do...,bafy2bzacebosq7iz7x5zkxshmlfhtivo7lyfjcv4phnfp...,bafy2bzacebum6yi6y4udbn77gsemasykcgojuqm7zjdcp...,2,gWNMKIPU6xCQdOpQAc1gCT6TWsiN2PHNAWgI73podubiTo...,2,hkqhgcx1Kii3qmxtn1OyS0jSIjXmeHXZwq5JDphairE3sL...
4,f02094335,"[{'Round': 3106804, 'Data': 'kVnCDgpqregXQEalF...","[{'PoStProof': 3, 'ProofBytes': 'rceLNSV0lbE5I...",[{'/': 'bafy2bzacedmrk3uc22dez5upjlisv4vo4blzm...,71041523330,3010959,1688635170,0,695136275,rf2ikzS6SZWQL/cPQFvnR8EcArZGYIO26guVuHA5qzeehY...,1,s4thMj3gFuTYZlpV0Pv0YesiPIDlAr65kAEnqbevem0Gd+...,bafy2bzaceadxsiykeh2neb2zjxqgaermn435rnmhoe7do...,bafy2bzacebosq7iz7x5zkxshmlfhtivo7lyfjcv4phnfp...,bafy2bzaceaba73bwgogz73br3ilivrn4w2mhh76niq2qw...,2,rHPZcNJtOyMInMyZqVHVjEIHKx3KOL2JU2e2HN0+uiM2nr...,2,hiBHH4LZhUL5GdUZOPsWBbkvgMnyBctt6WZ7FRnlcWAZ6O...


We can get all active miners.

In [6]:
state_miners = filecoin_rpc("Filecoin.StateListMiners", [chain_head["Cids"]])

In [7]:
print(f"Total number of miners is {len(state_miners)}")

Total number of miners is 599454


We could, in theory, get the power of each miner by calling Filecoin.StateMinerPower, but it is be very slow. 

In [68]:
# from tqdm.auto import tqdm

# mp = []
# for miner in tqdm(state_miners):
#     m = filecoin_rpc("Filecoin.StateMinerPower", [miner, chain_head["Cids"]])
#     m["MinerID"] = miner
#     mp.append(m)

# pd.json_normalize(mp).sample(5)


We can get messages from random blocks too!

In [8]:
block = chain_head["Cids"][0]

block_messages = filecoin_rpc("Filecoin.ChainGetBlockMessages", [block])
pd.json_normalize(block_messages["SecpkMessages"])

,Message.Version,Message.To,Message.From,Message.Nonce,Message.Value,Message.GasLimit,Message.GasFeeCap,Message.GasPremium,Message.Method,Message.Params,Message.CID./,Signature.Type,Signature.Data,CID./
0,0,f410fzdso6eki2epyyvl7m57ohrzzahgxs27wbwomb2i,f410fzopow76ttxe5rg2usuktqf44d7z4poutvcw55fy,21,1400000000000000000000,163654841,1000000000,60000000,3844450837,WETH5V8+AAAAAAAAAAAAAAAAy57rf9OdydibVJUVOBecH/...,bafy2bzacebsdmvortsutbudhjuaxqc7tytoofwj4ltzyj...,3,9QkosE5ywgNQ3v+gxO4ottUj7FIrY2/Rz51RB289ubAalj...,bafy2bzacedgph4zahh327psuhunmkl4yyfylhk5ik6ycf...
1,0,f410frb3y4qzq7wbtyvazewzsesrjxslxoyh5bkfjgqq,f410fe4uwmefaifwdztcg4shzq2mr7jft5qwlixz6vpy,43,0,513689639,835293041,56859492,3844450837,RE5x2S0=,bafy2bzacecbwvvpxursoqdpsvlnhcdjafcjk3zzak7uih...,3,QTikbF4MJl6Xr57CYhlZrdkA6EItxL6Nhz/Aejl6DhcxNC...,bafy2bzaceaje27c7gowmgitslihuf652ngucwo7qlgkpg...
2,0,f026868,f13sb4pa34qzf35txnan4fqjfkwwqgldz6ekh5trq,2062299,0,11674716,10000000000,56512856,2,hFUBN9PJi4OMRY3NEKGMn2LdLd2HCnxJADsdXqtXTL3hAEA=,bafy2bzacedbgoaymtfoyssjojv5jaeihyumfkgxpha4v6...,1,Nn5i8Z7w0+W86n+2roJOi8lZuNm3bcIVOq4xjiGFm0pgbT...,bafy2bzaceaef6gcrmphjeavowcdxiue4pfxgx2blqhwfg...
3,0,f026302,f13sb4pa34qzf35txnan4fqjfkwwqgldz6ekh5trq,2062300,0,11674716,10000000000,56512856,2,hFUBJLFiXJ0E4mTUTKtzJtTLETKAxcJJAAe58P64qARpAEA=,bafy2bzacecxfhk2tf7m5a3ojms6g2lt6m2g5q4nxwuurf...,1,PHa8ufAniiKO5NrTtWg4YImQNiJFlHlapMMWPy4MjlVfgk...,bafy2bzaceblj45rrwwrwbejbwuzdd6a4smnaf3azwip26...
4,0,f028566,f13sb4pa34qzf35txnan4fqjfkwwqgldz6ekh5trq,2062301,0,11674716,10000000000,56512856,2,hFUBl2VyTzC8iv5bWdNxnM/d1ilasyZJACxWBwCBeXziAEA=,bafy2bzacedci7bkcmnddgr4tvpdyqunkfuc4ohwfntovw...,1,m9hQpxn2y/BNb3bDoXT06wKXGOSRjnhUM5YDW7fkm5BZxJ...,bafy2bzacecofjvhoo7irmsa5hjjoxegiow2nopoaurxww...
5,0,f3rclxqsagkr5umn5x547hlo27pgsmxtoghbrbsl7rc4sr...,f1fvclkh7c6cqxcvo45imkuer2uu3ydlmubp7qoba,90,200000000000000000000,1388929,1218621813,56288445,0,None,bafy2bzaceda4hv65uto42bgcoda2zaxl3ekarp527riso...,1,e872GssPFGOUw3I+gFJO62Qb6c5neaAJ0wDJbUPBi7BAkF...,bafy2bzacec46lw4qxs2wvlezz7zikdmfldnicldbeipgx...
6,0,f410fukchwwhrgaqpho6ncfvb64yz23lk5mmaefr3sey,f410fqdngdps366gxchvyctqawtitg7tqekna5ropyjq,968,1000000000000000000,2159067,829612051,54124768,3844450837,None,bafy2bzacea43pgtucebgyodkknw25r5k6j5ggb3dy7hop...,3,A1Xxd3/hIZyoDsJIL1dh0ePaqVUiHVm5VqHrvHhqhutKSK...,bafy2bzacecdncukzweocbbbypluhv63drrkederoxb7so...


Messages from 5 heights ago:

In [9]:
pd.json_normalize(get_tipset_by_height(chain_head["Height"] - 5)["Blocks"])

,Miner,BeaconEntries,WinPoStProof,Parents,ParentWeight,Height,Timestamp,ForkSignaling,ParentBaseFee,Ticket.VRFProof,ElectionProof.WinCount,ElectionProof.VRFProof,ParentStateRoot./,ParentMessageReceipts./,Messages./,BLSAggregate.Type,BLSAggregate.Data,BlockSig.Type,BlockSig.Data
0,f0107916,"[{'Round': 3106799, 'Data': 'uQTZ5f3f8ehpZdY6f...","[{'PoStProof': 3, 'ProofBytes': 'uOUW9o0dhpvD9...",[{'/': 'bafy2bzaceb5sexdhybj54g3zmg32essqs7nbc...,71041420113,3010954,1688635020,0,522257319,lGy0Oc5FlJxASnhSRMKCSJuSG529GvIT8ATH9J83Azq9P9...,1,tqNro+l1De4aE32AjHOq4hQjyu/jVXn+IqQFKoFAdGelvR...,bafy2bzaceb3wop4g3oyfamv57dnwu4vjohmdabt2opay2...,bafy2bzaceacukbtbwurq2ttaapsqmrwgxwb3tsehmehi4...,bafy2bzacedil2assn4izt4nfkobhc7wib54phowwqdpyy...,2,oq3iz1UKuNiNK/UhXgmMnoHM8TtYAv0Vxvw/7ntE7oKAAk...,2,iGS/SNV7f1z0gfCCIVAeavNsGkbMu25q6gVCGgApeWzaSB...
1,f0442671,"[{'Round': 3106799, 'Data': 'uQTZ5f3f8ehpZdY6f...","[{'PoStProof': 4, 'ProofBytes': 'oGQUKAGp58OmU...",[{'/': 'bafy2bzaceb5sexdhybj54g3zmg32essqs7nbc...,71041420113,3010954,1688635020,0,522257319,k/tzRyooJXH2babrUPNf5pt4jhpmrKUbTWg/K94aPnxwek...,1,steC7st5X6sdzzqCkmBVvU9J4BCxN7GNROzLSZIy4hxSo9...,bafy2bzaceb3wop4g3oyfamv57dnwu4vjohmdabt2opay2...,bafy2bzaceacukbtbwurq2ttaapsqmrwgxwb3tsehmehi4...,bafy2bzacebhdaxvcwwlfqrg6xr72gcih5lyufcjzg36t5...,2,lPhURfLAvRClzXYbLV/aVUi+9si2Wvn91y/3KAG5tfGj2F...,2,qLXeJ9qFR+nPkHi7xlYTu0gJBN8xfdrULlkrYQUjocWGeS...
2,f0730266,"[{'Round': 3106799, 'Data': 'uQTZ5f3f8ehpZdY6f...","[{'PoStProof': 4, 'ProofBytes': 'sK0R33Gx3zxxK...",[{'/': 'bafy2bzaceb5sexdhybj54g3zmg32essqs7nbc...,71041420113,3010954,1688635020,0,522257319,mF38QRY1YQN06qTIaYWEGF02SH3x/eFsxnq9eySnX8h49K...,1,lhxa9a4VmX4DkeceXkmK/CVeuWsQ1MAb0onbIgOzeP6RR5...,bafy2bzaceb3wop4g3oyfamv57dnwu4vjohmdabt2opay2...,bafy2bzaceacukbtbwurq2ttaapsqmrwgxwb3tsehmehi4...,bafy2bzaceacemurbp5dwpnpb26kw67bvfcvf3fgk7dcbc...,2,g+L86WXk3I4COwy0lRQR/sjoYj3UfXhU/wDgOGH23yTtZu...,2,snD7KI1ahTRs2DBDZj926xCBHAkJW+43B0njiMJyUF+2C6...
3,f02037700,"[{'Round': 3106799, 'Data': 'uQTZ5f3f8ehpZdY6f...","[{'PoStProof': 3, 'ProofBytes': 'kJrPi6qGGkp4D...",[{'/': 'bafy2bzaceb5sexdhybj54g3zmg32essqs7nbc...,71041420113,3010954,1688635020,0,522257319,h6yA5aqcIb9LVltOufMoxdez5dMCKM/rT+yNDiN9uK454p...,1,oq1eYT8nlIgIt1rCe02UFR6H+9namtORKj8owlWOLqWVuD...,bafy2bzaceb3wop4g3oyfamv57dnwu4vjohmdabt2opay2...,bafy2bzaceacukbtbwurq2ttaapsqmrwgxwb3tsehmehi4...,bafy2bzacedfkgcmxoubhsufcfvlcdalqsipl5iuknr523...,2,j/R9tkOQI2staR/iXqcChArvPnYjFBBEvk82APrrDdcdV1...,2,hSFPV0O7uZD35SpiFhsw9amoFmkWzytbSvYYrF6+AUPleI...
